In [ ]:
pip install hazm

     |████████████████████████████████| 316 kB 7.5 MB/s 
     |████████████████████████████████| 1.4 MB 42.6 MB/s 
     |████████████████████████████████| 233 kB 51.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=3c70737dbfeff033509f28062474b4edc9ed99cbbc0fe6444c94113857da7913
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154702 sha256=3c4665a7e1745903ab5b2762c321f33fd33a24d7b703d6e7a45b9dfc49e3088b
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from __future__ import print_function, unicode_literals
import codecs, subprocess, random
from collections import Counter
from itertools import islice
from nltk.tag import untag
from sklearn.model_selection import train_test_split
from hazm import *
from hazm.Chunker import tree2brackets
from hazm.PeykareReader import coarse_pos_e as peykare_coarse_pos_e
from hazm.DadeganReader import coarse_pos_e as dadegan_coarse_pos_e
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import nltk
import glob
import re
import os
import io

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#stop word
import pandas as pd
stop_words = pd.read_json('/content/drive/MyDrive/Colab Notebooks/emotion_project/stopwords-fa.json', encoding= 'utf-8')
stopwords=stop_words.transpose()
stop_words=stopwords.values.tolist()
stopwords[1]='!!'
stop_words=stopwords.values.tolist()
ss=stopwords.iloc[0,9:798]
sss=ss.values.tolist()

In [ ]:
# clean data function
!pip install emojis
import emojis
import re
import string
from string import digits
def text_cleaner(text):

  # Emoji parsing
  text = emojis.decode(str(text))
  
  # Removing numbers
  text = re.sub(r'\\d+', '', text)

  # Removing single character words
  text =  re.sub(r"\b[a-zA-Z]\b", "", text)

  # Slang parsing
  # for slang, meaning in slangs.items():
  #   text = text.replace(slang, meaning)
  
  text=text.replace(":", "")
  text=text.replace('@', "")
  text=text.replace("#", " ")
  text=text.replace("_", " ")
  text=text.replace("،", " ")
  text=text.replace("\u200c", " ")
  text =  re.sub(r"[a-zA-Z]", "", text)
  
  # Removing links
  text=text.replace(r"http\S+", "")
  text=text.translate(str.maketrans('', '', string.punctuation))
  text=text.translate(str.maketrans('', '', string.digits))
  # text = text.translate(None, digits)

  # Normalizing Case
  # Filter Out Punctuation
  # Filter out Stop Words (and Pipeline)

  # words = [w for w in text.split() if w.isalpha() and not w in sss]
  
  words = [w for w in text.split()if not w in sss]
  return ' '.join(words)

In [ ]:
#clean data adn stemming
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/Emotion.csv', encoding= 'utf-8')
df.loc[0,'text']
clean_text=[]
stemmer = Stemmer()
for i in range(len(df)) :
  text=text_cleaner(df.loc[i,'text'])
  # text=df.loc[i,'text']
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text.append(t)
# clean_text[0]
len(clean_text)

5999

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/EmotionTest.csv', encoding= 'utf-8')
clean_text_test=[]
stemmer = Stemmer()
for i in range(len(df_test)) :
  text=text_cleaner(df_test.loc[i,'text'])
  # text=df.loc[i,'text']
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text_test.append(t)
len(clean_text_test)

1000

# implementaion of 
RW0 = "RW0"
RWeidf = "RWeidf"
RWeoc = "RWeoc"

In [ ]:
import os
import numpy as np
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn import svm
import spacy
import math
import time

RW0 = "RW0"
RWeidf = "RWeidf"
RWeoc = "RWeoc"

C_value = 0.95
D = 0.85
MIN_DIFF = 1e-5

MAX_FEATURES = 700
RW_MODE = RW0
WINDOW_SIZE = 4
STEPS = 2


def occurence_features(documents, dictionary):

    features_matrix = np.zeros((len(documents), len(dictionary)))

    # collecting the number of occurances of each of the words in the emails
    for doc_index, document in enumerate(documents):
        words = document.split()
        for word_index, word in enumerate(dictionary):
            features_matrix[doc_index,
                            word_index] = words.count(word)

    return features_matrix


def Dictionary_and_TF_IDF_feature(documents):

    vectorizer = TfidfVectorizer(max_features=MAX_FEATURES)
    # vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    tokens = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()

    return tokens, denselist


def TF_IDF_feature(documents, dictionary):

    # tf_matrix = np.zeros((len(documents), len(dictionary)))
    # idf_matrix = np.zeros((len(documents), len(dictionary)))
    tf_idf_matrix = np.zeros((len(documents), len(dictionary)))

    N = len(documents)
    size = len(dictionary)

    for word_index, word in enumerate(dictionary):
        print("test data TF-IDF :", round((word_index/size) * 100, 2), "%")

        df = 0
        occurance = 0
        for doc_index, document in enumerate(documents):
            words = document.split()
            occurance = words.count(word)
            # if occurance != 0:
            #     df += 1

            tf = occurance/(len(words)+1)
            idf = math.log(N/(df+1))

            # tf_matrix[doc_index, word_index] = tf
            # tf_matrix[doc_index, word_index] = idf
            tf_idf_matrix[doc_index, word_index] = tf * idf

    return tf_idf_matrix


def symmetrize(a):
    return a + a.T - np.diag(a.diagonal())


def get_token_pairs(window_size, document):
    """Build token_pairs from windows in sentences"""
    token_pairs = list()

    words = document.split()
    for i, word in enumerate(words):
        for j in range(i+1, i + window_size):
            if j >= len(words):
                break
            pair = (word, words[j])
            if pair not in token_pairs:
                token_pairs.append(pair)

    return token_pairs


def get_matrix(vocab, token_pairs, d_matrix, C):
    """Get normalized matrix"""

    # Build matrix
    vocab_size = len(vocab)
    g = np.zeros((vocab_size, vocab_size), dtype='float')
    for word1, word2 in token_pairs:
        if word1 in vocab and word2 in vocab:
            i, j = vocab.index(word1), vocab.index(word2)
            g[i][j] = 1

    # Get Symmeric matrix
    g = symmetrize(g)

    np.multiply(g, d_matrix)

    # all elements * C
    g * C

    # Normalize matrix by column
    norm = np.sum(g, axis=0)

    # # this is ignore the 0 element in norm
    g_norm = np.divide(g, norm, where=norm != 0)

    return g_norm


def rw_feature(documents, dictionary, E, RWmode):
    rw_matrix = np.zeros((len(documents), len(dictionary)))
    N = len(dictionary)
    size = len(documents)

    if RWmode == RW0:
        d_matrix = np.full((N, N), 1)
        C = 1

    for doc_index, document in enumerate(documents):
        print(RWmode + " featureing :", round((doc_index/size) * 100, 2), "%")

        # for RWeidf and RWeoc
        if RWmode != RW0:
            C = C_value
            E_vec = np.array([E[doc_index]])
            d_matrix = E_vec.transpose() * E_vec

            if RWmode == RWeidf:
                d_matrix / d_matrix.max()

        # Get token_pairs from windows
        token_pairs = get_token_pairs(WINDOW_SIZE, document)

        # Get normalized matrix
        g = get_matrix(dictionary, token_pairs, d_matrix, C)

        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(dictionary))

        # Iteration
        previous_pr = 0
        for epoch in range(STEPS):
            pr = (1-D) + D * np.dot(g, pr)

            if abs(previous_pr - sum(pr)) < MIN_DIFF:
                break
            else:
                previous_pr = sum(pr)

        # add value to all elements of pr
        if RWmode == RW0:
            pr = pr*D + (1-D)/N

        # for RWeidf and RWeoc
        else:
            pr + (1-D)/N

        rw_matrix[doc_index] = pr

    return rw_matrix



def TF_features(documents, dictionary):

    features_matrix = np.zeros((len(documents), len(dictionary)))

    # collecting the number of occurances of each of the words in the emails
    for doc_index, document in enumerate(documents):
        words = document.split()
        for word_index, word in enumerate(dictionary):
            features_matrix[doc_index,
                            word_index] = words.count(word)

        # normalize
        temp = features_matrix[doc_index]
        norm = np.linalg.norm(temp)
        features_matrix[doc_index] = temp/norm

    return features_matrix


start = time.time()

# train_dir = './LSpam/train_data'

# print('1. Train data pre process')
# train_data = preProccesing(train_dir, "train")
train_data=clean_text
print('2. Create Dictionary and Building training features and labels')
dictionary, tfidf = Dictionary_and_TF_IDF_feature(train_data)
# features_train = TF_features(train_data, dictionary)

features_train = rw_feature(train_data, dictionary, tfidf, RW_MODE)

# labels_train = build_labels(train_dir)

type(features_train)

Streaming output truncated to the last 5000 lines.
RW0 featureing : 16.65 %
RW0 featureing : 16.67 %
RW0 featureing : 16.69 %
RW0 featureing : 16.7 %
RW0 featureing : 16.72 %
RW0 featureing : 16.74 %
RW0 featureing : 16.75 %
RW0 featureing : 16.77 %
RW0 featureing : 16.79 %
RW0 featureing : 16.8 %
RW0 featureing : 16.82 %
RW0 featureing : 16.84 %
RW0 featureing : 16.85 %
RW0 featureing : 16.87 %
RW0 featureing : 16.89 %
RW0 featureing : 16.9 %
RW0 featureing : 16.92 %
RW0 featureing : 16.94 %
RW0 featureing : 16.95 %
RW0 featureing : 16.97 %
RW0 featureing : 16.99 %
RW0 featureing : 17.0 %
RW0 featureing : 17.02 %
RW0 featureing : 17.04 %
RW0 featureing : 17.05 %
RW0 featureing : 17.07 %
RW0 featureing : 17.09 %
RW0 featureing : 17.1 %
RW0 featureing : 17.12 %
RW0 featureing : 17.14 %
RW0 featureing : 17.15 %
RW0 featureing : 17.17 %
RW0 featureing : 17.19 %
RW0 featureing : 17.2 %
RW0 featureing : 17.22 %
RW0 featureing : 17.24 %
RW0 featureing : 17.25 %
RW0 featureing : 17.27 %
RW0 f

numpy.ndarray

In [ ]:
len(features_train)


5999

In [ ]:
y=df.loc[:,'label']
y.replace('\n','')
y=y.values.tolist()
# x=df_featurs.values.tolist()
for i in range(len(y)):
  y[i]=y[i].replace('\n','')
  y[i]=y[i].replace('.','')
label=[]
for i in range(len(y)):
  yy = [int(t) for t in y[i].split(' ')]
  label.append(yy)
label=np.array(label)

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/EmotionTest.csv', encoding= 'utf-8')
clean_text_test=[]
stemmer = Stemmer()
for i in range(len(df_test)) :
  text=text_cleaner(df_test.loc[i,'text'])
  # text=df.loc[i,'text']
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text_test.append(t)
len(clean_text_test)
y_test=df_test.loc[:,'label']
y_test.replace('\n','')
y_test=y_test.values.tolist()
# x_test=df_featurs2.values.tolist()
for i in range(len(y_test)):
  y_test[i]=y_test[i].replace('\n','')
  y_test[i]=y_test[i].replace('.','')

ytest=[]
for i in range(len(y_test)):
  yy = [int(t) for t in y_test[i].split(' ')]
  ytest.append(yy)
ytest=np.array(ytest)

In [ ]:
#classify
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
labels_train = label
classifier = MultiOutputClassifier(XGBClassifier())

clf = Pipeline([('classify', classifier)])


print('4. Test data pre process')

test_data = clean_text_test
print('5. Building the test features and labels')
# features_test = TF_features(test_data, dictionary)
tfidf = TF_IDF_feature(test_data, dictionary)
features_test = rw_feature(test_data, dictionary, tfidf, RW_MODE)

print('6. Training the classifier')
# nb_classifier.fit(features_train, labels_train)
# svm_classifier.fit(features_train, labels_train)
clf.fit(features_train, labels_train)

duration = round(time.time() - start, 4)


labels_test = ytest
print('7. Calculating accuracy of the trained classifier')
# nb_accuracy = nb_classifier.score(features_test, labels_test)
# svm_accuracy = svm_classifier.score(features_test, labels_test)
yhat = clf.predict(features_test)

auc_y1 = roc_auc_score(ytest[:,0],yhat[:,0])
auc_y2 = roc_auc_score(ytest[:,1],yhat[:,1])
auc_y3 = roc_auc_score(ytest[:,2],yhat[:,2])
auc_y4 = roc_auc_score(ytest[:,3],yhat[:,3])
auc_y5 = roc_auc_score(ytest[:,4],yhat[:,4])
auc_y6 = roc_auc_score(ytest[:,5],yhat[:,5])
auc_y7 = roc_auc_score(ytest[:,6],yhat[:,6])
auc_y8 = roc_auc_score(ytest[:,7],yhat[:,7])
auc_y9 = roc_auc_score(ytest[:,8],yhat[:,8])
auc_y10 = roc_auc_score(ytest[:,9],yhat[:,9])
print("ROC AUC y1: %.4f, y2: %.4f, y3: %.4f, y4: %.4f, y5: %.4f ,y6: %.4f, y7: %.4f, y8: %.4f, y9: %.4f, y10: %.4f" % (auc_y1, auc_y2, auc_y3, auc_y4, auc_y5,auc_y6, auc_y7, auc_y8, auc_y9, auc_y10))

4. Test data pre process
5. Building the test features and labels
test data TF-IDF : 0.0 %
test data TF-IDF : 0.14 %
test data TF-IDF : 0.29 %
test data TF-IDF : 0.43 %
test data TF-IDF : 0.57 %
test data TF-IDF : 0.71 %
test data TF-IDF : 0.86 %
test data TF-IDF : 1.0 %
test data TF-IDF : 1.14 %
test data TF-IDF : 1.29 %
test data TF-IDF : 1.43 %
test data TF-IDF : 1.57 %
test data TF-IDF : 1.71 %
test data TF-IDF : 1.86 %
test data TF-IDF : 2.0 %
test data TF-IDF : 2.14 %
test data TF-IDF : 2.29 %
test data TF-IDF : 2.43 %
test data TF-IDF : 2.57 %
test data TF-IDF : 2.71 %
test data TF-IDF : 2.86 %
test data TF-IDF : 3.0 %
test data TF-IDF : 3.14 %
test data TF-IDF : 3.29 %
test data TF-IDF : 3.43 %
test data TF-IDF : 3.57 %
test data TF-IDF : 3.71 %
test data TF-IDF : 3.86 %
test data TF-IDF : 4.0 %
test data TF-IDF : 4.14 %
test data TF-IDF : 4.29 %
test data TF-IDF : 4.43 %
test data TF-IDF : 4.57 %
test data TF-IDF : 4.71 %
test data TF-IDF : 4.86 %
test data TF-IDF : 5.0 %
test

In [ ]:
from sklearn.metrics import hamming_loss, accuracy_score
print(hamming_loss(ytest, yhat))

0.1186


In [ ]:
!pip install scikit-multilearn

In [ ]:
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score
mlknn_classifier = MLkNN()
mlknn_classifier.fit(features_train, labels_train)
yhat = mlknn_classifier.predict(features_test)
  
# print(accuracy_score(ytest, predicted))
print(hamming_loss(ytest, yhat))


0.1296


In [ ]:
yhat=yhat.toarray()
auc_y1 = roc_auc_score(ytest[:,0],yhat[:,0])
auc_y2 = roc_auc_score(ytest[:,1],yhat[:,1])
auc_y3 = roc_auc_score(ytest[:,2],yhat[:,2])
auc_y4 = roc_auc_score(ytest[:,3],yhat[:,3])
auc_y5 = roc_auc_score(ytest[:,4],yhat[:,4])
auc_y6 = roc_auc_score(ytest[:,5],yhat[:,5])
auc_y7 = roc_auc_score(ytest[:,6],yhat[:,6])
auc_y8 = roc_auc_score(ytest[:,7],yhat[:,7])
auc_y9 = roc_auc_score(ytest[:,8],yhat[:,8])
auc_y10 = roc_auc_score(ytest[:,9],yhat[:,9])
print("ROC AUC y1: %.4f, y2: %.4f, y3: %.4f, y4: %.4f, y5: %.4f ,y6: %.4f, y7: %.4f, y8: %.4f, y9: %.4f, y10: %.4f" % (auc_y1, auc_y2, auc_y3, auc_y4, auc_y5,auc_y6, auc_y7, auc_y8, auc_y9, auc_y10))

ROC AUC y1: 0.5044, y2: 0.5270, y3: 0.5445, y4: 0.5038, y5: 0.5509 ,y6: 0.5000, y7: 0.5231, y8: 0.5000, y9: 0.5324, y10: 0.5452
